In [1]:
# ✅ Step 1: Install KaggleHub and Download Dataset
!pip install kagglehub

import kagglehub

# Download dataset directly from Kaggle
path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")
print("Path to dataset files:", path)

100%|██████████| 105M/105M [00:00<00:00, 135MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images/versions/3


In [3]:


# ✅ Step 2: Import Libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# ✅ Step 3: Device Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# ✅ Step 4: Data Paths (Now using the dataset downloaded from Kaggle)
train_dir = path + '/train'  # Dataset train folder
test_dir = path + '/test'    # Dataset test folder

# ✅ Step 5: Data Augmentation and Normalization
transform = transforms.Compose([
    transforms.Resize((128, 128)),       # Upscale 32x32 to 128x128
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])   # ImageNet normalization
])

# ✅ Step 6: Load Dataset
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# ✅ Step 7: Model Setup
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: fake, real
model = model.to(device)

# ✅ Step 8: Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# ✅ Step 9: Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 0:  # Log every 100 iterations
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# ✅ Step 10: Evaluation
model.eval()
y_true = []
y_pred = []
y_prob = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)[:,1]
        preds = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

# ✅ Step 11: Metrics
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc = roc_auc_score(y_true, y_prob)

print(f'Accuracy: {acc:.4f}')
print(f'Precision: {prec:.4f}')
print(f'Recall: {rec:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'ROC-AUC: {roc:.4f}')


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [1/10], Step [0/3125], Loss: 0.0067
Epoch [1/10], Step [100/3125], Loss: 0.2656
Epoch [1/10], Step [200/3125], Loss: 0.1672
Epoch [1/10], Step [300/3125], Loss: 0.1512
Epoch [1/10], Step [400/3125], Loss: 0.1404
Epoch [1/10], Step [500/3125], Loss: 0.1396
Epoch [1/10], Step [600/3125], Loss: 0.1209
Epoch [1/10], Step [700/3125], Loss: 0.1176
Epoch [1/10], Step [800/3125], Loss: 0.1139
Epoch [1/10], Step [900/3125], Loss: 0.1147
Epoch [1/10], Step [1000/3125], Loss: 0.1204
Epoch [1/10], Step [1100/3125], Loss: 0.1068
Epoch [1/10], Step [1200/3125], Loss: 0.1065
Epoch [1/10], Step [1300/3125], Loss: 0.1022
Epoch [1/10], Step [1400/3125], Loss: 0.0882
Epoch [1/10], Step [1500/3125], Loss: 0.1102
Epoch [1/10], Step [1600/3125], Loss: 0.1050
Epoch [1/10], Step [1700/3125], Loss: 0.0920
Epoch [1/10], Step [1800/3125], Loss: 0.0988
Epoch [1/10], Step [1900/3125], Loss: 0.0932
Epoch [1/10], Step [2000/3125], Loss: 0.0912
Epoch [1/10], Step [2100/3125], Loss: 0.0981
Epoch [1/10], Step [22

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# 1. Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 10  # Can reduce to 2-3 for CPU testing

# 2. Data Preprocessing & Augmentation
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32 (adjust if needed)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 3. Load Dataset (ImageFolder for fake/real folders)
train_dir = r'D:\ML\minor project\deepfake image\train'
test_dir = r'D:\ML\minor project\deepfake image\test'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# 4. Simple 5-Layer CNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 1 * 1, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = self.pool(self.relu(self.conv5(x)))
        x = x.view(-1, 256 * 1 * 1)  # Flatten the output
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 5. Model, Loss, Optimizer
device = torch.device('cpu')  # Use CPU
model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 6. Training Loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# 7. Evaluation on Test Set
model.eval()
y_true = []
y_pred = []
y_prob = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)[:, 1]
        preds = torch.argmax(outputs, dim=1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_prob.extend(probs.cpu().numpy())

# 8. Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_prob)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')
print(f'ROC-AUC: {roc_auc:.4f}')


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001DA5A7BBF60>
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "C:\ProgramData\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py", line 1582, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\ProgramData\anaconda3\Lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\multiprocessing\popen_spawn_win32.py", line 112, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


KeyboardInterrupt: 